## 參考D097的程式碼，將訓練資料讀取方式改寫成 Generator，並將原本的 model.fit 改為 model.fit_generator 來進行訓練。

In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, SeparableConv2D
from keras.optimizers import RMSprop, Adam
import os

In [2]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## 自訂義generator

In [3]:
def cifar_generator(images,labels, batch_size=32):
  while True:
    for indexs in range(0, len(images), batch_size):
      batch_images = images[indexs: indexs+batch_size]
      batch_labels = labels[indexs: indexs+batch_size]
      yield batch_images, batch_labels

## 使用自訂義的 cifar_generator 配合 model.fit_generator 來訓練 CNN (注意, 此功能未來會被移除)

In [4]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

50000 train samples
10000 test samples
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit_generator(cifar_generator(x_train, y_train,batch_size=32),
                steps_per_epoch=len(x_train)/32,
                epochs=epochs,
                verbose=1,
                validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        1

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1562/1562 [==============================] - 11s 6ms/step - loss: 1.7552 - accuracy: 0.3581 - val_loss: 1.2057 - val_accuracy: 0.5746
Epoch 2/10
1562/1562 [==============================] - 8s 5ms/step - loss: 1.1629 - accuracy: 0.5915 - val_loss: 1.0355 - val_accuracy: 0.6406
Epoch 3/10
1562/1562 [==============================] - 8s 5ms/step - loss: 1.0330 - accuracy: 0.6471 - val_loss: 0.9913 - val_accuracy: 0.6646
Epoch 4/10
1562/1562 [==============================] - 8s 5ms/step - loss: 0.9964 - accuracy: 0.6621 - val_loss: 0.8628 - val_accuracy: 0.7049
Epoch 5/10
1562/1562 [==============================] - 8s 5ms/step - loss: 0.9977 - accuracy: 0.6697 - val_loss: 1.1125 - val_accuracy: 0.6459
Epoch 6/10
1562/1562 [==============================] - 8s 5ms/step - loss: 1.0218 - accuracy: 0.6648 - val_loss: 1.2696 - val_accuracy: 0.5934
Epoch 7/10
1562/1562 [==============================] - 8s 5ms/step - loss: 1.0666 - accuracy: 0.6532 - val_loss: 1.2114 - val_accuracy: 0.6368
Ep